# URL Classification with Support Vector Machine (SVM)
This notebook implements URL classification (malicious URL detection) using SVM algorithm.
It includes comprehensive feature engineering, feature scaling, and model evaluation.

## 1. Import Libraries

In [5]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from urllib.parse import urlparse
from tld import get_tld
import os.path

# ML libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.metrics import confusion_matrix
import joblib
from wordcloud import WordCloud

# Cuda core
from thundersvm import SVC as ThunderSVC

# import warnings
# warnings.filterwarnings('ignore')

# CONFIG
DATA_PATH = "../artifacts/dataset/data.csv"
ATTATCH_PATH="./attach"
APP_NAME='Malsy Gate'


OSError: libcusparse.so.9.0: cannot open shared object file: No such file or directory

## 2. Load Dataset

In [12]:
# Load the Dataset
df = pd.read_csv(DATA_PATH)
print(f"Dataset shape: {df.shape}")
print(f"\nDataset columns: {df.columns.tolist()}")
print(f"\nDataset head:")
df.head()

Dataset shape: (651191, 2)

Dataset columns: ['url', 'type']

Dataset head:


,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [13]:
# Check data types and null values
print("Data Info:")
df.info()
print("\nNull values:")
print(df.isnull().sum())
print("\nTarget distribution:")
print(df['type'].value_counts())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB

Null values:
url     0
type    0
dtype: int64

Target distribution:
type
benign        428103
defacement     96457
phishing       94111
malware        32520
Name: count, dtype: int64


In [ ]:
plt.figure(figsize= (10 , 5))
plt.title('Distribution of Types of Attacks')
sns.countplot(x = 'type', data = df)
# ax.bar_label(ax.containers[0])
plt.xlabel('Attacks Type')

plt.savefig(ATTATCH_PATH + '/type_of_accatck.png', bbox_inches='tight')

### WordCloud of Different Types of Attacks

In [ ]:
df_phish = df[df['type'] == 'phishing']
df_benign = df[df['type'] == 'benign']
df_deface = df[df['type'] == 'defacement']
df_malware = df[df['type'] == 'malware']

**Phishing Url**

In [ ]:
phish_url = " ".join(i for i in df_phish.url)
# print(phish_url[0: 1000])
wordcloud = WordCloud(
    width=1600, 
    height=800, 
    background_color="white",  
    colormap="Set2", 
    
).generate(phish_url)


plt.figure( figsize=(16,18),facecolor='black')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
plt.savefig(ATTATCH_PATH + '/phish_wordcloud.png', bbox_inches='tight')

**Benign Url**

In [ ]:
benign_url = " ".join(i for i in df_benign.url)

wordcloud = WordCloud(
    width=1600, 
    height=800, 
    background_color="white",  
    colormap="Set2", 
    
).generate(benign_url)


plt.figure( figsize=(16,18),facecolor='black')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
plt.savefig(ATTATCH_PATH + '/benign_wordcloud.png')

**Defracment Url**

In [ ]:
deface_url = " ".join(i for i in df_deface.url)

wordcloud = WordCloud(
    width=1600, 
    height=800, 
    background_color="white",  
    colormap="Set2", 
    
).generate(deface_url)


plt.figure( figsize=(16,18),facecolor='black')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
plt.savefig(ATTATCH_PATH + '/deface_wordcloud.png')

**Malware**

In [ ]:
malware_url = " ".join(i for i in df_malware.url)

wordcloud = WordCloud(
    width=1600, 
    height=800, 
    background_color="white",  
    colormap="Set2", 
    
).generate(malware_url)


plt.figure( figsize=(16,18),facecolor='black')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()
plt.savefig(ATTATCH_PATH + '/malware_wordcloud.png')

## 3. Feature Extraction

extract the following lexical features from raw URLs, as these features will be used as the input features for training the machine learning model. The following features are created as follows:

### 3.1 IP Address Detection

In [ ]:
# Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    
    if match:
        return 1
    else:
        return 0

df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))
print(df['use_of_ip'].value_counts())

In [ ]:
df[df['use_of_ip'] == 1]

In [ ]:
plt.figure(figsize = (8 , 6))
sns.set(style="darkgrid")
ax = sns.countplot(x="type", data=df, hue="use_of_ip")
plt.xlabel('Types of Attacks')
plt.title(APP_NAME + " - Usage of Ip Address")

**Shape data**

In [ ]:
arr = np.where(df[df['type'] == 'benign']['use_of_ip'] == 1)
arr[0].shape

In [ ]:
arr = np.where(df[df['type'] == 'malware']['use_of_ip'] == 1)
arr[0].shape

In [ ]:
arr = np.where(df[df['type'] == 'phishing']['use_of_ip'] == 1)
arr[0].shape

In [ ]:
arr = np.where(df[df['type'] == 'defacement']['use_of_ip'] == 1)
arr[0].shape

### 3.2 Abnormal URL

In [ ]:
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:      
        return 1
    else:
        return 0

df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))
print(df['abnormal_url'].value_counts())

In [ ]:
plt.figure(figsize = (8 , 6))
sns.set(style="darkgrid")
ax = sns.countplot(x="type", data=df, hue="abnormal_url")
plt.xlabel('Types of Attacks')
plt.title(APP_NAME + "- Usage of Ip Address in domain name in with Diffrent types of attacks")

**Shape data**

In [ ]:
arr = df[df['type'] == 'benign']['abnormal_url'].value_counts()
arr[0].shape
# arr

In [ ]:
arr = df[df['type'] == 'phishing']['abnormal_url'].value_counts()
arr[0].shape
# arr

In [ ]:
arr = df[df['type'] == 'defacement']['abnormal_url'].value_counts()
arr[0].shape
# arr

In [ ]:
arr = df[df['type'] == 'malware']['abnormal_url'].value_counts()
arr[0].shape
# arr

### 3.3 Google Index

In [ ]:
def google_index(url):
    
    site = search(url, 5)
    return 1 if site else 0

df['google_index'] = df['url'].apply(lambda i: google_index(i))

In [ ]:
print(df['url'][0])
for j in search(df['url'][0], num_results=10):
    print(j)

In [ ]:
df['google_index'].value_counts()

### 3.4 Count Features

In [ ]:
# Count dot (.)
def count_dot(url):
    count_dot = url.count('.')
    return count_dot

df['count_.'] = df['url'].apply(lambda i: count_dot(i))

# Count www
def count_www(url):
    url.count('www')
    return url.count('www')

df['count_www'] = df['url'].apply(lambda i: count_www(i))

# Count @
def count_atrate(url):
    return url.count('@')

df['count_@'] = df['url'].apply(lambda i: count_atrate(i))

# Count directories
def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))

# Count embedded domains
def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

df['count_embed-domain'] = df['url'].apply(lambda i: no_of_embed(i))



**Count .**

In [ ]:
plt.figure(figsize = (22 , 100))
sns.set(style="darkgrid")
ax = sns.countplot(y="count_.", data=df)
plt.ylabel('No. of Dots in URL')
plt.title(APP_NAME + " - No. Of Dots Available in URLS ")

### 3.5 Suspicious Words Detection

In [18]:
def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0
    
df['sus_url'] = df['url'].apply(lambda i: suspicious_words(i))

### 3.6 URL Shortening Service Detection

In [19]:
def shortening_service(url):
    match = re.search(r'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      r'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      r'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      r'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      r'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      r'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      r'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      r'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
    
df['short_url'] = df['url'].apply(lambda i: shortening_service(i))

### 3.7 Protocol Counts

In [20]:
# Count https
def count_https(url):
    return url.count('https')

df['count_https'] = df['url'].apply(lambda i: count_https(i))

# Count http
def count_http(url):
    return url.count('http')

df['count_http'] = df['url'].apply(lambda i: count_http(i))

### 3.8 Special Character Counts

In [21]:
# Count %
def count_per(url):
    return url.count('%')

df['count%'] = df['url'].apply(lambda i: count_per(i))

# Count ?
def count_ques(url):
    return url.count('?')

df['count?'] = df['url'].apply(lambda i: count_ques(i))

# Count -
def count_hyphen(url):
    return url.count('-')

df['count-'] = df['url'].apply(lambda i: count_hyphen(i))

# Count =
def count_equal(url):
    return url.count('=')

df['count='] = df['url'].apply(lambda i: count_equal(i))

### 3.9 Length Features

In [22]:
# URL length
def url_length(url):
    return len(str(url))

df['url_length'] = df['url'].apply(lambda i: url_length(i))

# Hostname length
def hostname_length(url):
    return len(urlparse(url).netloc)

df['hostname_length'] = df['url'].apply(lambda i: hostname_length(i))

### 3.10 Domain Features

In [23]:
# First directory length
def fd_length(url):
    urlpath = urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

df['fd_length'] = df['url'].apply(lambda i: fd_length(i))

# TLD length
def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1

df['tld'] = df['url'].apply(lambda i: get_tld(i, fail_silently=True))
df['tld_length'] = df['tld'].apply(lambda i: tld_length(i))
df = df.drop("tld", axis=1)

### 3.11 Character Counts

In [24]:
# Count digits
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

df['count_digits'] = df['url'].apply(lambda i: digit_count(i))

# Count letters
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

df['count_letters'] = df['url'].apply(lambda i: letter_count(i))

## 4. Label Encoding

In [25]:
# Label Encoding
label_encoder = LabelEncoder()
df['type_code'] = label_encoder.fit_transform(df['type'])

print("Label mapping:")
for i, label in enumerate(label_encoder.classes_):
    print(f"{label}: {i}")

print("\nTarget distribution after encoding:")
print(df['type_code'].value_counts())

Label mapping:
benign: 0
defacement: 1
malware: 2
phishing: 3

Target distribution after encoding:
type_code
0    428103
1     96457
3     94111
2     32520
Name: count, dtype: int64


## 5. Feature and Target Creation

In [ ]:
# Create feature matrix X and target vector Y
X = df[['use_of_ip','abnormal_url', 'count_.', 'count_www', 'count_@',
       'count_dir', 'count_embed_domain', 'short_url', 'count%', 'count?', 
       'count-', 'count=', 'url_length', 'count_https', 'count_http', 
       'hostname_length', 'sus_url', 'fd_length', 'tld_length', 'count_digits',
       'count_letters']]

Y = df['type_code']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {Y.shape}")
print(f"\nFeatures: {X.columns.tolist()}")

Feature matrix shape: (651191, 21)
Target vector shape: (651191,)

Features: ['use_of_ip', 'abnormal_url', 'count.', 'count-www', 'count@', 'count_dir', 'count_embed_domain', 'short_url', 'count%', 'count?', 'count-', 'count=', 'url_length', 'count_https', 'count_http', 'hostname_length', 'sus_url', 'fd_length', 'tld_length', 'count_digits', 'count_letters']


## 6. Train-Test Split

In [27]:
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"\nTraining target distribution:")
print(Y_train.value_counts())
print(f"\nTest target distribution:")
print(Y_test.value_counts())

Training set size: (520952, 21)
Test set size: (130239, 21)

Training target distribution:
type_code
0    342325
1     77353
3     75275
2     25999
Name: count, dtype: int64

Test target distribution:
type_code
0    85778
1    19104
3    18836
2     6521
Name: count, dtype: int64


## 7. Feature Scaling
SVM requires feature scaling for optimal performance.

In [28]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data (using the same scaler fitted on training data)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed!")
print(f"Scaled training set shape: {X_train_scaled.shape}")
print(f"Scaled test set shape: {X_test_scaled.shape}")

Feature scaling completed!
Scaled training set shape: (520952, 21)
Scaled test set shape: (130239, 21)


## 8. SVM Model

In [ ]:
# svm_model = SVC(
#     kernel='rbf',
#     C=1.0,
#     probability=True,
#     random_state=42
# )

svm_model = ThunderSVC(
    kernel='rbf',
    C=1.0,
    # default gpu = 0: primary gpu, 1: secondary gpu
    gpu_id=0,
    probability=True,
    random_state=42
)

print("Training SVM model...")
print("This may take a few minutes...")
svm_model.fit(X_train_scaled, Y_train)
print("Training complete!")

Training SVM model...
This may take a few minutes...


: 

: 

## 9. Model Evaluation

In [ ]:
# Predictions
Y_pred_train = svm_model.predict(X_train_scaled)
Y_pred_test = svm_model.predict(X_test_scaled)

# Metrics
train_accuracy = accuracy_score(Y_train, Y_pred_train) * 100
test_accuracy = accuracy_score(Y_test, Y_pred_test) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Detailed classification report
print("\n" + "="*50)
print("Classification Report (Test Set)")
print("="*50)
print(classification_report(Y_test, Y_pred_test, target_names=label_encoder.classes_))

## 10. Confusion Matrix

In [ ]:
# Create confusion matrix
cm = confusion_matrix(Y_test, Y_pred_test)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix - SVM', fontsize=16)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.show()

# Print confusion matrix values
print("Confusion Matrix:")
print(cm)

## 11. Model Export

In [ ]:
# Save the model
model_filename = 'best_svm_model.pkl'
joblib.dump(svm_model, model_filename)
print(f"Model saved as: {model_filename}")

# Save the scaler
scaler_filename = 'svm_scaler.pkl'
joblib.dump(scaler, scaler_filename)
print(f"Scaler saved as: {scaler_filename}")

# Save the label encoder
encoder_filename = 'label_encoder_svm.pkl'
joblib.dump(label_encoder, encoder_filename)
print(f"Label encoder saved as: {encoder_filename}")

# Save model performance metrics
metrics = {
    'model': 'SVM',
    'train_accuracy': train_accuracy,
    'test_accuracy': test_accuracy,
    'kernel': 'rbf',
    'C': 1.0,
    'random_state': 42
}

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('svm_model_metrics.csv', index=False)
print(f"Metrics saved to: svm_model_metrics.csv")

## 12. Summary

In [ ]:
print("="*60)
print("SVM MODEL SUMMARY")
print("="*60)
print(f"Algorithm: Support Vector Machine")
print(f"Kernel: RBF")
print(f"Regularization (C): 1.0")
print(f"Number of Features: {X.shape[1]}")
print(f"Training Samples: {X_train.shape[0]}")
print(f"Test Samples: {X_test.shape[0]}")
print(f"\nPerformance:")
print(f"  - Training Accuracy: {train_accuracy:.2f}%")
print(f"  - Test Accuracy: {test_accuracy:.2f}%")
print(f"\nModel saved as: {model_filename}")
print(f"Scaler saved as: {scaler_filename}")
print("="*60)